<a href="https://colab.research.google.com/github/kathdevx/emotionalAI/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Separation of training files from the dataset

with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files.txt', 'w') as outfile:
    with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/meta-info/Partitions.txt', 'r') as part_file:
        for line in part_file:
            idx = line.find(';')
            if line[:idx] == 'Train':
                outfile.write(line[idx+2:-1])
                outfile.write('\n')

In [28]:
train_fnames = []

with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files.txt', 'r') as infile:
    train_fnames = infile.read().splitlines()

Kapoia arxeia den vrethikan

In [22]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

print(file_len('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files_path.txt'))

43588


In [ ]:
dir = '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/Audio Files/'

import soundfile as sf
import glob

train_data = open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files_path.txt', 'w')

for filename in glob.glob(dir+'*.wav'):
    idx = filename.rfind('/')
    fname = filename[idx+1:]

    if fname in train_fnames:
        train_data.write(fname)
        train_data.write(' ')
        train_data.write(filename)
        train_data.write('\n')

train_data.close()

# Begin **HERE**

In [ ]:
!pip install soundfile
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-nmlnhthr
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-nmlnhthr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import librosa
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import nltk

def load_model():
    tokenizer = Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec2-large-960h')
    model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-large-960h')
    return tokenizer, model

def correct_sentence(input_text):
    sentences = nltk.sent_tokenize(input_text)
    return (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

def asr_transcript(input_file):

    tokenizer, model = load_model();
    
    speech, fs = sf.read(input_file)

    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]

    if fs != 16000:
        speech = librosa.resample(speech, fs, 16000)

    input_values = tokenizer(speech, return_tensors="pt").input_values;
    logits = model(input_values).logits;
    
    predicted_ids = torch.argmax(logits, dim=-1)
    
    transcription = tokenizer.decode(predicted_ids[0]);

    return correct_sentence(transcription.lower())

In [ ]:
def chunks(lst, n):
    chunky = []
    for i in range(0, len(lst), n):
        chunky.append(lst[i:i + n])
    return chunky

In [23]:
batch_size = 50
train_data = []

with open('/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/train_files_path.txt', 'r') as tdata:
    data = tdata.read().splitlines()
    train_data = chunks(data, batch_size)

# Don't uncomment, files already created

In [25]:
# import pickle 

# for i in range(len(train_data)):
#     fname = '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/training chunks/train_chunk_' + str(i) + '.pkl'
#     with open(fname, 'wb') as outfile:
#         pickle.dump(train_data[i],outfile)

In [37]:
katerina = range(0, 436)
xristina = range(436, 873)
global_counter = xristina[0]

In [ ]:
while global_counter <= xristina[-1]:
    print(f'----------- Epoch {global_counter} ----------- \n')
    inname = '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/training chunks/train_chunk_' + str(global_counter) + '.pkl'
    outname = '/content/drive/Shareddrives/DNN -  audio files/Podcasts_Dataset/Data/training sentences/train_sentences_chunk' + str(global_counter) + '.txt'


    train_chunk = None
    with open(inname, 'rb') as infile:
        train_chunk = pickle.load(infile)
    with open(outname, 'w') as outfile:
        for t in train_chunk: 
            temp = t.split(' ', 1)
            fname = temp[0] + ' '
            outfile.write(fname)
            outfile.write(asr_transcript(temp[1]))
            outfile.write('\n')
    global_counter +=1
    break